In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import difflib

In [2]:
df = pd.read_json('Fix1.json')
df.sort_index(inplace=True)

In [3]:
# دانشگاه‌های پذیرفته شده: accUni
# دانشگاه انتخاب شده:apUni
# Renaming : Cause it is not clear
# + Make them a same style!
df.rename(columns={"apUni": 'uniSelected', 'accUni': 'uniAccepted','rejUni':'uniRejected','masterUni':'uniMasters','bachelorsUni':'uniBachelors'}, inplace=True)
df.to_json('Data_Uni.json',date_format='utf8')

In [4]:
col=['uniSelected','uniMasters','uniBachelors']
for c in col:
    df[c].replace(r'\-+','n/a',inplace=True,regex=True)
col.append('uniAccepted')
col.append('uniRejected')
for c in col:
    df[c].replace(r'\-+','n/a',inplace=True,regex=True)
    for i in df.index:
        df.set_value(i, c, df.ix[i][c].replace('u of','').replace('university of','')
                     .replace('uni of','').replace('university','').replace('uni ','')
                     .replace('universidad de','').replace('universidad','').strip())

# Preparing 9k Uni name 

In [5]:
cdf=pd.read_csv('../10kUni.csv',names=['country','name','site'])
cdf.name=cdf.name.str.lower()
cdf.head()

,country,name,site
0,AD,university of andorra,http://www.uda.ad/
1,AE,abu dhabi university,http://www.adu.ac.ae/
2,AE,ajman university of science & technology,http://www.ajman.ac.ae/
3,AE,alain university of science and technology,http://www.alainuniversity.ac.ae/
4,AE,al ghurair university,http://www.agu.ae/


In [6]:
for i in cdf.index:
    s=cdf.ix[i]['site'].split('.')[1]
    cdf.set_value(i, 'site',s)
    w=cdf.ix[i]['name'].replace(' and ','').replace('&','').replace('university of','').replace('university','').replace('technology','').replace('science','').replace('universidad de','').replace('universidad','').strip()
    cdf.set_value(i, 'name',w)
cdf.head()

,country,name,site
0,AD,andorra,uda
1,AE,abu dhabi,adu
2,AE,ajman,ajman
3,AE,alain,alainuniversity
4,AE,al ghurair,agu


### Type of Uni

In [7]:
for c in col:
    print(c,df[c].value_counts().count())

('uniSelected', 1168)
('uniMasters', 601)
('uniBachelors', 1148)
('uniAccepted', 2313)
('uniRejected', 1202)


### Changing uni.name with length less than 5 char

### just counting

In [8]:
# rm Bachelors , because they are mostly iranian uni
del col[2]

In [9]:
for c in col:
    counter=0
    for i in df.index:
        if (len(df.ix[i][c])<5):counter+=1
    print(str(counter)+' records with <5Chars in '+ str(c))

1012 records with <5Chars in uniSelected
2007 records with <5Chars in uniMasters
812 records with <5Chars in uniAccepted
2280 records with <5Chars in uniRejected


In [10]:
#get part before ',' ... to handle multivalue records
for c in col:
    for i in df.index:
        df.set_value(i, c,(df.ix[i][c]).split(',')[0])
for i in cdf.index:
    cdf.set_value(i, c,(cdf.ix[i]['name']).split(',')[0])

In [109]:
for c in col:
    counter=0
    for i in df.index:
        if (len(df.ix[i][c])<5):counter+=1
    print(str(counter)+' records with <5Chars in '+ str(c))

1022 records with <5Chars in uniSelected
2011 records with <5Chars in uniMasters
1041 records with <5Chars in uniAccepted
2446 records with <5Chars in uniRejected


In [12]:
cdf.to_csv('../10kUni_Cleaned.csv',date_format='utf8',columns=['country','name','site'])

## Filling

# 1st wave

In [111]:
for c in col:
    counter=0
    for i in df.index:
        if (len(df.ix[i][c])<5) & (df.ix[i][c]!='n/a'):
            isin=cdf['site'].isin([df.ix[i][c]])
            isin=isin[isin==True]
            if len(isin)==0:
                continue
            inx=isin.index[0]
            res=cdf.ix[inx]['name']
            df.set_value(i, c,str.strip(res))
            counter+=1
    print(str(counter)+' records changed in '+ str(c))

487 records changed in uniSelected
159 records changed in uniMasters
354 records changed in uniAccepted
196 records changed in uniRejected


In [112]:
for c in col:
    counter=0
    for i in df.index:
        if (len(df.ix[i][c])<5):counter+=1
    print(str(counter)+' records with <5Chars in '+ str(c))

562 records with <5Chars in uniSelected
1856 records with <5Chars in uniMasters
706 records with <5Chars in uniAccepted
2267 records with <5Chars in uniRejected


# 2nd wave

In [113]:
for c in col:
    counter=0
    for i in df.index:
        if (len(df.ix[i][c])<5) & (df.ix[i][c]!='n/a'):
            r=difflib.get_close_matches(df.ix[i][c],cdf.site)
            if len(r)>0:
                res=cdf[cdf['site']==r[0]]
                df.set_value(i, c,str.strip(res.name.values[0]))
                counter+=1
    print(str(counter)+' records changed in '+ str(c))

120 records changed in uniSelected
29 records changed in uniMasters
91 records changed in uniAccepted
52 records changed in uniRejected


In [114]:
for c in col:
    counter=0
    for i in df.index:
        if (len(df.ix[i][c])<5):counter+=1
    print(str(counter)+' records with <5Chars in '+ str(c))

484 records with <5Chars in uniSelected
1832 records with <5Chars in uniMasters
641 records with <5Chars in uniAccepted
2233 records with <5Chars in uniRejected


### Counting Changes

In [115]:
print("Old Records:\n('uniSelected', 1168)\n('uniMasters', 601)\n('uniAccepted', 2313)\n('uniRejected', 1202)")
print '\nNew records:'
for c in col:
    print(c,df[c].value_counts().count())

Old Records:
('uniSelected', 1168)
('uniMasters', 601)
('uniAccepted', 2313)
('uniRejected', 1202)

New records:
('uniSelected', 1109)
('uniMasters', 590)
('uniAccepted', 1655)
('uniRejected', 801)


### Func to check txt is Persian or not

In [116]:
import re
def checkFarsi(w):
    regexp = re.compile(r'[a-zA-Z?]')
    return (not regexp.search(w))

### Try to fit Bigger words 

#### Counting words need to change

In [131]:
notin=[]
for c in col:
    arenotin=[]
    for i in df.index:
        temp=df.ix[i][c]
        if checkFarsi(temp) |(len(temp)<5) :continue
        isin=cdf['name'].isin([df.ix[i][c]])
        isin=isin[isin==True]
        if len(isin)==0:
            arenotin.append(i)
    print(len(arenotin),len(df))
    notin.append(arenotin)

(1197, 3526)
(386, 3526)
(1664, 3526)
(703, 3526)


In [133]:
for key,c in enumerate(col):
    counter=0
    for i in notin[key]:
        temp=df.ix[i][c]
        if checkFarsi(temp) |(len(temp)<5) :continue
        r=difflib.get_close_matches(temp,cdf.name)
        if len(r)>0:
            res=cdf[cdf['name']==r[0]]
            df.set_value(i, c,str.strip(res.name.values[0]))
            counter+=1
    print(str(counter)+' records changed in '+ str(c))

1047 records changed in uniSelected
315 records changed in uniMasters
1192 records changed in uniAccepted
514 records changed in uniRejected


In [134]:
for c in col:
    print(c,df[c].value_counts().count())

('uniSelected', 825)
('uniMasters', 505)
('uniAccepted', 1171)
('uniRejected', 607)


In [135]:
df.to_json('Fix2.json',date_format='utf8')